# Installing neccessary packages


# Install your packages here

In [1]:
install.packages('jsonlite')
install.packages('rvest')
install.packages('tidyverse')
install.packages('lubridate')
install.packages('httr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("jsonlite"):
“installation of package ‘jsonlite’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘reprex’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("httr"):
“installation of package ‘httr’ had non-zero exit status”


# Load the libraries here

In [2]:
library(jsonlite)
library(httr)
library(rvest)
library(tidyverse)
library(lubridate)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ purrr::flatten()        masks jsonlite::flatten()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




# trimStringArray function

In [3]:
trimStringArray = function(array) {
  if (is.array(array)) {
    for (i in 1:length(array())) {
      array[i] = trimws(array[i])
    }
  } else {
    array = trimws(array)
  }
  array
}

# ternaryOperatorFunction | Vectorized function

In [4]:
ternary = function(expression_1, expression_2, condition) {
    if(condition) {
        expression_1
    } else {
        expression_2
    }
}

ternary_v = Vectorize(ternary)

# Create a vectorized version of strsplit()

In [5]:
strsplit_custom = function (string, splitter, index) {
    strsplit(string, splitter)[[1]][index]
}

strsplit_custom_v = Vectorize(strsplit_custom)

# Scrape Teilnehmer von WM

In [6]:
scrapeColumnsWM = function(game_name) {

  # base url

  url = 'https://www.transfermarkt.com/weltmeisterschaft-2022/teilnehmer/pokalwettbewerb/WM22/saison_id/2021'

  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header
  slugs = c()
  club_ids = c()

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    item %>%
      html_elements('.table-header') %>%
      html_text() %>%
      trimStringArray() -> name

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # scrape the url with the slugs and club ids of the countries

      responsive_table %>%
          html_elements('tr td:first-of-type a') %>%
          html_attr('href') %>%
          strsplit('/') -> raw_url_data
      
      raw_url_data

      # obtain the slug of all countries

      raw_url_data[1][[1]][2]
      raw_url_data[1][[1]][length(raw_url_data[1][[1]])]

      for (i in 1:length(raw_url_data)) {
          slugs = c(slugs, raw_url_data[i][[1]][2])
          club_ids = c(club_ids, raw_url_data[i][[1]][length(raw_url_data[1][[1]])])
      }

      # print slugs

      slugs

      # print club_ids

      club_ids

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames

      # add slug and club_id to columnNames

      columnNames = c(columnNames, c('slug', 'club_id'))

      # print columnNames again

      columnNames

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              print(paste0("Column ", i, " is not a valid value"))
          }

      }

      # add club_id and slug to values

      columnValues = append(columnValues, list(slugs))
      columnValues = append(columnValues, list(club_ids))

      # print columnValues

      columnValues

      # return list
      list(columnNames, columnValues)
  } else {
      # return NULL
      NULL
  }
}

# create Dataframe from scrapeColumnsWM

In [7]:
create_dataframeWM = function(name) {
  df_wm = scrapeColumnsWM(name)

  if(!(is.null(df_wm))) {
      columns = df_wm[[2]]
      columnNames = df_wm[[1]]

      #print(columnNames)

      df_wm = data.frame(columns[[1]])

      for (i in 2:length(columns)) {
          df_wm[i] = columns[[i]]
      }

      names(df_wm) = columnNames[2:length(columnNames)]
      df_wm
  } else {
      NULL
  }
}

In [8]:
df_wm = create_dataframeWM('teilnehmende teams an der wm')
df_wm

[1] "Column 1 is not a valid value"


Club,Squad,&oslash-Age,WC particip.,Foreigners,Market Value,&oslash-Market Value,slug,club_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
England,27,25.6,16,11.1 %,€1.29bn,€47.89m,england,3299
France,25,26.9,16,76.0 %,€1.06bn,€42.40m,frankreich,3377
Brazil,26,27.2,22,88.5 %,€963.00m,€37.04m,brasilien,3439
Germany,25,26.6,20,28.0 %,€824.50m,€32.98m,deutschland,3262
Portugal,24,27.3,8,87.5 %,€772.00m,€32.17m,portugal,3300
Spain,24,25.9,16,29.2 %,€751.00m,€31.29m,spanien,3375
Argentina,32,27.7,18,84.4 %,€695.00m,€21.72m,argentinien,3437
Netherlands,25,26.7,11,48.0 %,€494.40m,€19.78m,niederlande,3379
Uruguay,41,26.9,14,58.5 %,€478.40m,€11.67m,uruguay,3449


In [9]:
teilnehmer = df_wm$Club
teilnehmer

[1] "England"       "France"        "Brazil"        "Germany"      
 [5] "Portugal"      "Spain"         "Argentina"     "Netherlands"  
 [9] "Uruguay"       "Serbia"        "Croatia"       "Belgium"      
[13] "Senegal"       "Denmark"       "Poland"        "Switzerland"  
[17] "Morocco"       "United States" "Ghana"         "Canada"       
[21] "Cameroon"      "Wales"         "Mexico"        "Ecuador"      
[25] "Australia"     "Tunisia"       "Saudi Arabia"  "Japan"        
[29] "South Korea"   "Qatar"         "Costa Rica"    "Iran"

# Scrape Columns
We wrote a function, that takes a url to scrape all tables

In [10]:
scrapeGame = function(year, country, club_id, game_name) {

  # base url

  url = paste0("https://www.transfermarkt.com/",
               country,
               "/spielplan/verein/",
               club_id,
               "/plus/0?saison_id=", year)
    
  # print url
  print(url)
    
  # get status code
  res = httr::GET(url)

  # read response
  code = httr::status_code(res)
    
  # print status code
  print(code)
    
  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    if(length(html_elements(item, '.table-header') %>% html_children())!= 0) {
      item %>%
        html_elements('.table-header a img') %>%
        html_attr('title') -> name

      #print(name)
    } else {
      item %>%
        html_elements('.table-header') %>%
        html_text() %>%
        trimStringArray() -> name
        
        #print(name)
    }

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames
      
      # add game_name to columnNames
      
      columnNames = append(columnNames, 'game_name')
      columnNames = append(columnNames, 'country')

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              #print(paste0("Column ", i, " is not a valid value"))
          }

      }
        
      # add game_name to columnValues
      
      columnValues = append(columnValues, name)
      
      # add country name to columnValues
      columnValues = append(columnValues, country)
      
      # return data frame
      # change column names by replacing whitespace with underscore
      columnNames = gsub(" ", "_", columnNames)

      # create a dataframe with first colum and append the rest (workaround)
      df = data.frame(columnValues[[1]])

      for (i in 2:length(columnValues)) {
          df[i] = columnValues[[i]]
      }

      names(df) = columnNames
      return(df)
      
  } else {
      return(data.frame())
  }
}

# Testing Section

In [31]:
#scrapeGame(year = '2009', country = 'England', club_id = 3299, game_name = "weltmeisterschaft [0-9]+")

scrapeGameByYear = function(year, country) {
    
    df_wm %>%
        filter(Club == country) -> tempDf
    
    tempDf$club_id -> club_id
    
    #pmap_df(list('2015', 'England', '3299', c('weltmeisterschaft [0-9]+', 'freundschaftsspiele', 'em-qualifikation', 'wm-qualifikation europa', 'europameisterschaft [0-9]+')), scrapeGame)
    return(pmap_df(list(year, country, club_id,
                        c(
                            #'world cup [0-9]+',
                            #'international friendlies'
                            #'european qualifiers',
                            #'world cup qualification europe'
                            #'euro [0-9]+'
                            #'copa'
                            'confederations cup [0-9]+'
                            #'ostasienmeisterschaft',
                            #'wm-qualifikation asien',
                            #'asienmeisterschaft [0-9]+',
                            #'asienmeisterschaft qualifikation'
                            
                            #'fifa arab cup',
                            #'gold cup [0-9]+'
                        )), scrapeGame))
}

In [32]:
scrapeSomething = function(country) {
    return(map2_df(seq('2009', '2021', by=1), country, scrapeGameByYear))
}

## Scrape world cup & friendlies for all 32 countries

In [34]:
complete_df = map_df(teilnehmer, scrapeSomething)

[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2009"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2010"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2011"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2012"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2013"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2014"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2015"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2016"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2017"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2018"
[1] 200
[1] "https://www.transfermarkt.com/Engla

ERROR: Error in open.connection(x, "rb"): Empty reply from server


In [29]:
complete_df

Matchday,Date,Time,Venue,Ranking,Opponent,System_of_play,Attendance,Result,game_name,country
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Group B,"Sun Jul 3, 2011",9:00 PM,H,,Venezuela,4-3-3 Defending,45.000,0:0,Copa América 2011,Brazil
Group B,"Sat Jul 9, 2011",9:00 PM,H,,Paraguay,4-3-3 Attacking,57.000,2:2,Copa América 2011,Brazil
Group B,"Thu Jul 14, 2011",2:45 AM,H,,Ecuador,4-2-3-1,39.000,4:2,Copa América 2011,Brazil
Quarter-Finals,"Sun Jul 17, 2011",9:00 PM,H,,Paraguay,4-2-3-1,36.000,0:2 on pens,Copa América 2011,Brazil
Group C,"Sun Jun 14, 2015",11:30 PM,H,,Peru,4-3-3 Attacking,16.342,2:1,Copa América 2015,Brazil
Group C,"Thu Jun 18, 2015",2:00 AM,H,,Colombia,4-3-3 Attacking,44.008,0:1,Copa América 2015,Brazil
Group C,"Sun Jun 21, 2015",11:30 PM,H,,Venezuela,4-2-3-1,33.383,2:1,Copa América 2015,Brazil
Quarter-Finals,"Sat Jun 27, 2015",11:30 PM,H,,Paraguay,4-2-3-1,29.276,4:5 on pens,Copa América 2015,Brazil
Group B,"Sun Jun 5, 2016",4:00 AM,H,,Ecuador,4-1-4-1,53.158,0:0,Copa América Centenario 2016,Brazil


# Split Result in Goals

In [16]:
#df_football %>%
#    mutate(goals_scored_england = ternary_v(strsplit_custom_v(Ergebnis, ':', 1), strsplit_custom_v(Ergebnis, ':', 2), (Heimmannschaft=='England'))) -> df_football_goals
#df_football_goals

## We save all data for the WorldCups&friendlies as csv

In [30]:
#complete_df
write.csv2(complete_df, file='./football_copa_america&america_centenario&centroamericana.csv', row.names = F)

## Modelling - Data Cleaning and Transformation

In [34]:
data_set = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/world_cup%26friendlies_football_en.csv', sep=';')
fifa_ranking = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")


In [6]:
fifa_ranking %>%
    separate(Rang, c('Rang', 'Sym')) %>%
    select(-'Sym') %>%
    mutate(fifa_date = ymd(Datum)) %>%
    select(-c('Datum')) -> fifa_ranking1
fifa_ranking1

date_list = unique(fifa_ranking1$fifa_date)
date_list

Land,Rang,Punkte,fifa_date
<chr>,<chr>,<chr>,<date>
Spain,1,1588,2009-09-02
Netherlands,2,1376,2009-09-02
Germany,3,1186,2009-09-02
Italy,4,1186,2009-09-02
Russia,5,1129,2009-09-02
England,6,1127,2009-09-02
Argentina,7,1113,2009-09-02
Croatia,8,1101,2009-09-02
France,9,1040,2009-09-02


[1] "2009-09-02" "2009-10-16" "2009-11-20" "2009-12-16" "2010-02-03"
  [6] "2010-03-03" "2010-03-31" "2010-04-28" "2010-05-26" "2010-07-14"
 [11] "2010-08-11" "2010-09-24" "2010-10-20" "2010-11-17" "2010-12-15"
 [16] "2011-01-12" "2011-02-02" "2011-03-09" "2011-04-13" "2011-05-18"
 [21] "2011-06-29" "2011-07-27" "2011-08-24" "2011-09-21" "2011-10-19"
 [26] "2011-11-23" "2011-12-21" "2012-01-18" "2012-02-15" "2012-03-07"
 [31] "2012-04-11" "2012-05-09" "2012-06-06" "2012-07-04" "2012-08-08"
 [36] "2012-09-05" "2012-10-03" "2012-11-07" "2012-12-19" "2013-01-17"
 [41] "2013-02-14" "2013-03-14" "2013-04-11" "2013-05-09" "2013-06-06"
 [46] "2013-07-04" "2013-08-08" "2013-09-12" "2013-10-17" "2013-11-28"
 [51] "2013-12-19" "2014-01-16" "2014-02-13" "2014-03-13" "2014-04-10"
 [56] "2014-05-08" "2014-07-17" "2014-08-14" "2014-09-18" "2014-10-23"
 [61] "2014-11-27" "2014-12-18" "2015-02-12" "2015-03-12" "2015-04-09"
 [66] "2015-05-07" "2015-06-04" "2015-07-09" "2015-08-06" "2015-09-03"
 [71] "2015-10-01" "2015-11-05" "2015-12-03" "2016-01-07" "2016-02-04"
 [76] "2016-03-03" "2016-04-07" "2016-05-05" "2016-05-20" "2016-06-02"
 [81] "2016-07-14" "2016-08-11" "2016-09-15" "2016-10-20" "2016-11-24"
 [86] "2016-12-22" "2017-01-12" "2017-02-09" "2017-03-09" "2017-04-06"
 [91] "2017-05-04" "2017-06-01" "2017-07-06" "2017-08-10" "2017-09-14"
 [96] "2017-10-16" "2017-11-23" "2017-12-21" "2018-01-18" "2018-02-15"
[101] "2018-03-15" "2018-04-12" "2018-05-17" "2018-06-07" "2018-08-16"
[106] "2018-09-20" "2018-10-25" "2018-11-29" "2018-12-20" "2019-02-08"
[111] "2019-04-04" "2019-07-25" "2019-09-10" "2019-09-11" "2019-10-24"
[116] "2019-11-28" "2020-02-20" "2020-03-24" "2020-07-23" "2020-09-17"
[121] "2020-10-22" "2020-11-27" "2020-12-10" "2021-02-03" "2021-02-18"
[126] "2021-06-15" "2021-08-12" "2021-10-20" "2021-10-21" "2021-11-19"
[131] "2021-12-23" "2022-02-10" "2022-03-31" "2022-06-06" "2022-06-23"

In [36]:
data_set

Matchday,Date,Time,Venue,Ranking,Opponent,System_of_play,Attendance,Result,game_name,country
<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Group C,"Sat Jun 12, 2010",8:30 PM,H,NA,United States,4-4-2 double 6,38.646,1:1,World Cup 2010,England
Group C,"Fri Jun 18, 2010",8:30 PM,H,NA,Algeria,4-4-2 double 6,64.100,0:0,World Cup 2010,England
Group C,"Wed Jun 23, 2010",4:00 PM,A,NA,Slovenia,4-4-2 double 6,36.893,0:1,World Cup 2010,England
Round of 16,"Sun Jun 27, 2010",4:00 PM,A,NA,Germany,4-4-2 double 6,40.510,4:1,World Cup 2010,England
NA,"Wed Mar 3, 2010",9:00 PM,H,NA,Egypt,4-4-2 double 6,80.602,3:1,International Friendlies,England
NA,"Mon May 24, 2010",9:00 PM,H,NA,Mexico,4-2-3-1,88.638,3:1,International Friendlies,England
NA,"Sun May 30, 2010",2:15 PM,H,NA,Japan,4-4-2 double 6,15.326,2:1,International Friendlies,England
NA,"Wed Aug 11, 2010",9:00 PM,H,NA,Hungary,4-2-3-1,72.024,2:1,International Friendlies,England
NA,"Wed Nov 17, 2010",9:00 PM,H,NA,France,4-2-3-1,85.495,1:2,International Friendlies,England


In [37]:
data_set_reduced = data_set %>%
    select(c('Date', 'Venue', 'Opponent', 'Result', 'game_name', 'country'))

In [46]:
lubridate::as_date('Jun 12, 2010')
lubridate:as_datetime('Jun 12, 2010')

Warning message:
“All formats failed to parse. No formats found.”


[1] NA

ERROR: Error in eval(expr, envir, enclos): object 'lubridate' not found


## Obtain goals of enemy and country

In [42]:
data_set_reduced %>%
    separate(Result, c('result_left', 'result_right')) %>%
    mutate(goals_country = ternary_v(result_left, result_right, Venue=='H')) %>%
    mutate(goals_enemy = ternary_v(result_left, result_right, Venue=='A')) -> data_set_increased

data_set_increased

Warning message:
“Expected 2 pieces. Additional pieces discarded in 38 rows [51, 53, 212, 304, 307, 420, 474, 531, 533, 534, 589, 617, 618, 660, 693, 832, 833, 839, 840, 841, ...].”


Date,Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Sat Jun 12, 2010",H,United States,1,1,World Cup 2010,England,1,1
"Fri Jun 18, 2010",H,Algeria,0,0,World Cup 2010,England,0,0
"Wed Jun 23, 2010",A,Slovenia,0,1,World Cup 2010,England,1,0
"Sun Jun 27, 2010",A,Germany,4,1,World Cup 2010,England,1,4
"Wed Mar 3, 2010",H,Egypt,3,1,International Friendlies,England,3,1
"Mon May 24, 2010",H,Mexico,3,1,International Friendlies,England,3,1
"Sun May 30, 2010",H,Japan,2,1,International Friendlies,England,2,1
"Wed Aug 11, 2010",H,Hungary,2,1,International Friendlies,England,2,1
"Wed Nov 17, 2010",H,France,1,2,International Friendlies,England,1,2


In [66]:
convert_month_to_decimal = function(month) {
    num = 0
    
    for (i in 1:12) {
        if(month == switch(i,
                          'Jan',
                          'Feb',
                          'Mar',
                          'Apr',
                          'May',
                          'Jun',
                          'Jul',
                          'Aug',
                          'Sep',
                          'Oct',
                          'Nov',
                          'Dec')
          ) {
            num = i
            break;
        }
    }
    return(i)
}

convert_month_to_decimal_v = Vectorize(convert_month_to_decimal)

In [85]:
data_set_increased %>%
    separate(Date, c('weekday', 'month', 'day', 'year'), sep=' ', remove=F) %>%
    separate(day, c('day')) %>%
    mutate(month = convert_month_to_decimal_v(month)) %>%
    mutate(date_of_play = ymd(paste0(year, '-', month, '-', day))) %>%
    select(-c('Date', 'weekday', 'month', 'day', 'year')) -> data_set_cleaned1

data_set_cleaned1

Warning message:
“Expected 1 pieces. Additional pieces discarded in 2399 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy,date_of_play
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>
H,United States,1,1,World Cup 2010,England,1,1,2010-06-12
H,Algeria,0,0,World Cup 2010,England,0,0,2010-06-18
A,Slovenia,0,1,World Cup 2010,England,1,0,2010-06-23
A,Germany,4,1,World Cup 2010,England,1,4,2010-06-27
H,Egypt,3,1,International Friendlies,England,3,1,2010-03-03
H,Mexico,3,1,International Friendlies,England,3,1,2010-05-24
H,Japan,2,1,International Friendlies,England,2,1,2010-05-30
H,Hungary,2,1,International Friendlies,England,2,1,2010-08-11
H,France,1,2,International Friendlies,England,1,2,2010-11-17


In [86]:
test_date = function(date_param) {
    for(i in 1:length(date_list)) {
        if(date_list[i] > date_param) {
            break;
        }
    }
    if(i != 1) {
        return(date_list[i-1])
    } else {
        return(date_list[1])
    }
}

test_date_v = Vectorize(test_date)

In [87]:
test = test_date(ymd('2013-06-02'))
test

[1] "2013-05-09"

In [88]:
data_set_cleaned1 %>%
    mutate(fifa_date = as_date(test_date_v(date_of_play))) -> data_set_fifa_date

In [89]:
data_set_fifa_date

Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy,date_of_play,fifa_date
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>
H,United States,1,1,World Cup 2010,England,1,1,2010-06-12,2010-05-26
H,Algeria,0,0,World Cup 2010,England,0,0,2010-06-18,2010-05-26
A,Slovenia,0,1,World Cup 2010,England,1,0,2010-06-23,2010-05-26
A,Germany,4,1,World Cup 2010,England,1,4,2010-06-27,2010-05-26
H,Egypt,3,1,International Friendlies,England,3,1,2010-03-03,2010-03-03
H,Mexico,3,1,International Friendlies,England,3,1,2010-05-24,2010-04-28
H,Japan,2,1,International Friendlies,England,2,1,2010-05-30,2010-05-26
H,Hungary,2,1,International Friendlies,England,2,1,2010-08-11,2010-08-11
H,France,1,2,International Friendlies,England,1,2,2010-11-17,2010-11-17


## join both data frames using fifa_date

In [90]:
merge(data_set_fifa_date, fifa_ranking1, by.x=c('country', 'fifa_date'), by.y=c('Land', 'fifa_date')) -> newest_df

In [109]:
newest_df %>%
    select(-c('Punkte')) -> final_data_set

unique(final_data_set$country)
final_data_set

[1] "Argentina"     "Australia"     "Belgium"       "Brazil"       
 [5] "Cameroon"      "Canada"        "Costa Rica"    "Croatia"      
 [9] "Denmark"       "Ecuador"       "England"       "France"       
[13] "Germany"       "Ghana"         "Iran"          "Japan"        
[17] "Mexico"        "Morocco"       "Netherlands"   "Poland"       
[21] "Portugal"      "Qatar"         "Saudi Arabia"  "Senegal"      
[25] "Serbia"        "South Korea"   "Spain"         "Switzerland"  
[29] "Tunisia"       "United States" "Uruguay"       "Wales"

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,Rang
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,8
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,8
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,9
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,7
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,7
Argentina,2010-05-26,H,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,7
Argentina,2010-05-26,H,South Korea,4,1,World Cup 2010,4,1,2010-06-17,7
Argentina,2010-05-26,A,Greece,0,2,World Cup 2010,2,0,2010-06-22,7
Argentina,2010-05-26,H,Mexico,3,1,World Cup 2010,3,1,2010-06-27,7


## Write function that maps a fifa ranking to each date of data_set_cleaned1

In [ ]:
data_set_cleaned1

## Combine fifa ranking and data_set_increased


In [5]:
#bind_cols(data_set_increased, fifa_ranking1)